In [ ]:
from helper import*
# list of sql variables
hostName = "localhost"
userName = "root"
userPassword = "1A3g5m7t9n#" # IMPORTANT! Put your MySQL Terminal password here.
databaseName = 'music_therapy_eeg'
tableName = 'MH_01_ES1_task1'
query = ("% s % s"%('SELECT * FROM', tableName))

In [ ]:
# list of python variables
fs = 256
collection_time = 120
fs_setting = 'resample'
plotTitles = ['RAW_TP9','RAW_AF7','RAW_AF8','RAW_TP10']
figSize = (10,10)
pltColor = ['r','g','b','k']

In [ ]:
# establish connection with music_therapy_eeg database
db_connection = create_db_connection(hostName,userName,userPassword,databaseName)
data = df_to_table_query(db_connection,query)

In [ ]:
# transform sql extracted table to raw eeg data
rawEEG = (transformToRawEEG(data,fs,collection_time,fs_setting))[0]
time_s = (transformToRawEEG(data,fs,collection_time,fs_setting))[1]
plots(time_s,rawEEG,plotTitles,figSize,pltColor)